***vis_completables_scores* function**:
* Input: players_info dictionary 
* Displays bar chart showing the score achieved by players in the different completables

In [1]:
def vis_completables_scores (players_info):
    completables=set()
    for player in players_info.keys():
        if players_info[player]["completables_scores"] != []:
            completables = completables.union(players_info[player]["completables_scores"].keys())
    completables = list(completables)

    # scores of each player per completable
    players_scores = []
    for player in players_info.keys():
        if players_info[player]["completables_scores"] != []:
            player_scores = []
            for c in completables:
                if c in players_info[player]["completables_scores"].keys():
                    height = players_info[player]["completables_scores"][c]
                else:
                    height = 0
                player_scores.append(height)
            players_scores.append(player_scores)
            
    # bar chart showing the score achieved by players in the different completables

    n_groups = len(completables)
    index = np.arange(n_groups)
    bar_width = 0.2
    opacity = 0.8
    if n_groups>0:
        for i in range(len(players_scores)):
            if len(players_scores[i])>0 :
                plt.bar(index+(i*bar_width), players_scores[i], bar_width, 
                        label=list(players_info.keys())[i], 
                        tick_label=completables)
        plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        plt.xlabel("Completable")
        plt.ylabel("Score")
        plt.xticks(rotation=90)
        plt.title("Scores of players per completable")
        plt.show()
    else:
        print("There is no data to fill a visualization.")